In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = "/Users/michalmacbookpro/Downloads/titanic"
df_train = pd.read_csv(path + "/train.csv")
df_test = pd.read_csv(path + "/test.csv")
submision = pd.read_csv(path + "/gender_submission.csv")

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer

In [4]:
#df_train = df_train[df_train["Fare"].between(0,100)]
#df_train = df_train[df_train["Age"].between(0,65)]

In [5]:
num_cols = Pipeline([("Imputer",SimpleImputer(strategy="median")),("Trans",QuantileTransformer())])

In [6]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder

In [7]:
cat_cols = Pipeline([
    ("Encoder",OrdinalEncoder()),
    ("Imputer",SimpleImputer(strategy="most_frequent")),
    ("OneHot",OneHotEncoder(sparse=False))
    ])

In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
df_train["Family"] = df_train["SibSp"] + df_train["Parch"]
#df_train.sample(10)
def cabinizer(df):
    df["Cabin_L"] = df["Cabin"].str.extract(r"(^\D{0,1})")
    df["Cabin_L"].fillna("U",inplace=True)
    df["Cabin_L"] = df["Cabin_L"].map({"A":"ABC","B":"ABC","C":"ABC","D":"DE","E":"DE","F":"FG","G":"FG","T":"ABC","U":"U"})
    return df

cabinizer(df_train)

def titulizer(df):
    df["Title"] = df["Name"].str.extract(r"([A-Za-z]+\.)")
    df["Title"].replace(['Capt.', 'Col.', 'Countess.', 'Don.', 'Dona.', 'Dr.', 'Jonkheer.', 'Lady.','Major.', 'Rev.', 'Sir.'], 'Rare.', inplace = True)
    df["Title"].replace(['Mlle.', 'Ms.'], 'Miss.', inplace = True)
    df["Title"].replace(['Mme.'], 'Mrs.', inplace = True)
    return df

titulizer(df_train)
df_train.sample()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Cabin_L,Title
208,209,1,3,"Carr, Miss. Helen ""Ellen""",female,16.0,0,0,367231,7.75,NaN,Q,0,U,Miss.


In [10]:
num = ["Age","Fare","Family"]
cat = ["Pclass","Sex","Embarked","Cabin_L","Title"]

In [11]:
preprocesing_pipeline = ColumnTransformer([
    ("num",num_cols,num),
    ("cat", cat_cols,cat)
])

In [12]:
from sklearn.model_selection import train_test_split

In [37]:
y = df_train.Survived
x = df_train.drop("Survived",axis=1)
#print(f"{X.shape}{y.shape}")

In [33]:
X = preprocesing_pipeline.fit_transform(x)
y = df_train.Survived

/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  warnings.warn(


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier


rfc = RandomForestClassifier()
dtc = DecisionTreeClassifier()

/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [17]:
param = [{
    "criterion": ['gini', 'entropy'],
    "n_estimators":[100,200,300,400,500,600],
    'max_depth': range(4, 20, 2),
    'max_leaf_nodes': range(4, 28, 2),
}]

gs_rt = RandomizedSearchCV(rfc, param, cv = 5, n_jobs = -1, verbose = 1)
gs_rt.fit(X_train, y_train)
y_pred = gs_rt.predict(X_test)
#print(gs_rt.best_estimator_)
print(accuracy_score(y_pred,y_test))
rt_rs_best = gs_rt.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.8208955223880597


In [18]:
param = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(4, 20, 2),
        'max_leaf_nodes': range(4, 28, 2),
    }
]

gs_dt = GridSearchCV(dtc, param, cv = 5, n_jobs = -1, verbose = 1)
gs_dt.fit(X_train, y_train)
y_pred = gs_dt.predict(X_test)
print(accuracy_score(y_pred,y_test))
dt_gs_best = gs_dt.best_estimator_
#print(f1_score(y_pred,y_test))

Fitting 5 folds for each of 192 candidates, totalling 960 fits
0.8246268656716418


In [19]:
from sklearn.svm import SVC

svc = SVC()

param = [
    {
        'kernel': ['poly'], 'C': [0.1, 0.3, 1, 2, 3, 4], 
        'gamma': [0.1,0.2,0.3]
    }, 
]

svc = SVC(probability = True)
rs_svc = RandomizedSearchCV(svc, param, cv = 5, n_jobs = -1, verbose = 1)
rs_svc.fit(X_train, y_train)
y_pred = rs_svc.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(rs_svc.best_estimator_)
print(accuracy_score(y_pred,y_test))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.8097014925373134
SVC(C=0.3, gamma=0.3, kernel='poly', probability=True)
0.8097014925373134


In [20]:
param = [
    {
        'kernel': ["poly"], 'C': [0.1, 0.3, 1, 2, 4], 
        'gamma': [0.1,0.3, 1, 3, 10, 12]
    }, 
]
#
#gs_svc = GridSearchCV(svc, param, cv = 5, n_jobs = -1, verbose = 1)
#svc = SVC(probability = True)
#gs_svc.fit(X_train, y_train)
#y_pred = gs_svc.predict(X_test)
print(accuracy_score(y_pred,y_test))
#print(gs_svc.best_estimator_)

#svc_best = gs_svc.best_estimator_
#print(svc_best)

0.8097014925373134


In [21]:
param = [
    {'n_estimators': range(8, 28, 4), 
     'max_depth': range(24, 48, 4),
     'max_leaf_nodes': range(20, 48, 4),
    }
]

et = ExtraTreesClassifier()
gs_et = RandomizedSearchCV(et, param, cv = 5, n_jobs = -1, verbose = 1)
gs_et.fit(X_train, y_train)
y_pred = gs_et.predict(X_test)
print(accuracy_score(y_pred,y_test))
gs_et_best = gs_et.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.835820895522388


In [22]:
param_grid={
    'max_depth': range(2, 10, 2),
    'n_estimators': range(26, 46, 2),
    'learning_rate': [0.2, 0.1, 0.03, 0.01]
}

xg = XGBClassifier(eval_metric='logloss', n_jobs = -1, use_label_encoder = False)
gs_xg = RandomizedSearchCV(xg, param_grid, cv = 5, n_jobs = -1, verbose = 1)
gs_xg.fit(X_train, y_train)
y_pred = gs_xg.predict(X_test)
print(accuracy_score(y_pred,y_test))
gs_xg_best = gs_xg.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype inste

0.8097014925373134


In [23]:
cabinizer(df_test)
titulizer(df_test)
df_test["Family"] = df_test["SibSp"] + df_test["Parch"]
df_test_test = preprocesing_pipeline.transform(df_test)

In [24]:
#y_pred_test = gs_dt.predict(df_test_test).astype("int")

In [25]:
#submision["Survived"] = y_pred_test
#submision.to_csv('submission.csv', index = False)

In [26]:
#y_pred = gs_svc.predict(df_test_test).astype("int")
#submision["Survived"] = y_pred
#submision.to_csv('submission.csv', index = False)

In [27]:
from sklearn.ensemble import VotingClassifier

In [28]:
###eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
#eclf1 = eclf1.fit(X, y)
#print(eclf1.predict(X))###

voting_clf = VotingClassifier(estimators=[
    ("rand_for",rt_rs_best),
    ("dec_tree",dt_gs_best),
    #("svc",svc_best),
    ("extree",gs_et_best),
    ("xgb",gs_xg_best)],voting = 'hard', n_jobs = -1, verbose = 1)
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))


[Voting] ................. (2 of 4) Processing dec_tree, total=   0.0s
[Voting] ................... (3 of 4) Processing extree, total=   0.0s
[Voting] ...................... (4 of 4) Processing xgb, total=   0.1s
[Voting] ................. (1 of 4) Processing rand_for, total=   0.4s
0.8246268656716418


In [29]:
submision["Survived"] = voting_clf.predict(df_test_test).astype("int")
submision.to_csv('submission.csv', index = False)

In [30]:
#y_pred=gs_xg_best.predict(df_test_test).astype("int")
#submision["Survived"] = y_pred
#submision.to_csv('submission.csv', index = False)

In [31]:
#sns.histplot(x=df_train["Fare"],hue=df_train["Survived"],multiple="dodge")